<a href="https://colab.research.google.com/github/Shreyes45/BIS-lab/blob/main/BIS-lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import math

class ACO_TSP:
    def __init__(self, cities, n_ants=10, n_iterations=100, alpha=1, beta=5, rho=0.5, Q=100):
        self.cities = cities
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q
        self.n_cities = len(cities)
        self.dist_matrix = self.calc_distances()
        self.pheromone = np.ones((self.n_cities, self.n_cities))

    def calc_distances(self):
        dist = np.zeros((self.n_cities, self.n_cities))
        for i in range(self.n_cities):
            for j in range(self.n_cities):
                dist[i][j] = math.hypot(self.cities[i][0] - self.cities[j][0], self.cities[i][1] - self.cities[j][1])
        return dist

    def route_length(self, route):
        return sum(self.dist_matrix[route[i]][route[(i+1)%self.n_cities]] for i in range(self.n_cities))

    def probability(self, i, unvisited):
        pheromone = np.array([self.pheromone[i][j] for j in unvisited])
        heuristic = np.array([1/self.dist_matrix[i][j] if self.dist_matrix[i][j] != 0 else 0 for j in unvisited])
        probs = (pheromone ** self.alpha) * (heuristic ** self.beta)
        return probs / probs.sum()

    def construct_solution(self):
        routes = []
        lengths = []
        for _ in range(self.n_ants):
            route = [random.randint(0, self.n_cities - 1)]
            unvisited = set(range(self.n_cities)) - {route[0]}
            while unvisited:
                i = route[-1]
                probs = self.probability(i, list(unvisited))
                next_city = random.choices(list(unvisited), weights=probs)[0]
                route.append(next_city)
                unvisited.remove(next_city)
            routes.append(route)
            lengths.append(self.route_length(route))
        return routes, lengths

    def update_pheromone(self, routes, lengths):
        self.pheromone *= (1 - self.rho)
        for r, L in zip(routes, lengths):
            for i in range(self.n_cities):
                a, b = r[i], r[(i+1)%self.n_cities]
                self.pheromone[a][b] += self.Q / L
                self.pheromone[b][a] += self.Q / L

    def run(self):
        best_route = None
        best_length = float('inf')
        for _ in range(self.n_iterations):
            routes, lengths = self.construct_solution()
            self.update_pheromone(routes, lengths)
            min_length = min(lengths)
            if min_length < best_length:
                best_length = min_length
                best_route = routes[lengths.index(min_length)]
        return best_route, best_length

cities = [(random.uniform(0,100), random.uniform(0,100)) for _ in range(10)]
aco = ACO_TSP(cities)
best_route, best_length = aco.run()
print("Best Route:", best_route)
print("Best Length:", best_length)


Best Route: [3, 9, 5, 8, 4, 6, 0, 2, 7, 1]
Best Length: 252.89334034635635
